In [21]:
# imports
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import re  
import emoji
from sklearn.utils.class_weight import compute_class_weight  # Missing import!



In [8]:
# load datasets
train_df = pd.read_csv('/home/chris/Documents/Sentiment-Analysis/Datasets/twitter_training.csv')
val_df =pd.read_csv('/home/chris/Documents/Sentiment-Analysis/Datasets/twitter_validation.csv')

# Basic exploration
print("Training set shape:", train_df.shape)
print("Validation set shape:", val_df.shape)
print("\nTraining set info:")
print(train_df.info())
print("\nClass distribution in training:")
# First, let's see what columns you actually have
print("Column names in training set:")
print(train_df.columns.tolist())
print("\nFirst few rows of training set:")
print(train_df.head())

print("\nColumn names in validation set:")
print(val_df.columns.tolist())
print("\nFirst few rows of validation set:")
print(val_df.head())


Training set shape: (74681, 4)
Validation set shape: (999, 4)

Training set info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74681 entries, 0 to 74680
Data columns (total 4 columns):
 #   Column                                                 Non-Null Count  Dtype 
---  ------                                                 --------------  ----- 
 0   2401                                                   74681 non-null  int64 
 1   Borderlands                                            74681 non-null  object
 2   Positive                                               74681 non-null  object
 3   im getting on borderlands and i will murder you all ,  73995 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.3+ MB
None

Class distribution in training:
Column names in training set:
['2401', 'Borderlands', 'Positive', 'im getting on borderlands and i will murder you all ,']

First few rows of training set:
   2401  Borderlands  Positive  \
0  2401  Borderlands  Positive   


In [9]:
# Check for missing values
print("\nMissing values in training set:")
print(train_df.isnull().sum())

print("\nMissing values in validation set:")
print(val_df.isnull().sum())

# Check data types
print("\nTraining set dtypes:")
print(train_df.dtypes)


Missing values in training set:
2401                                                       0
Borderlands                                                0
Positive                                                   0
im getting on borderlands and i will murder you all ,    686
dtype: int64

Missing values in validation set:
3364                                                                                                                                                                                                                                                  0
Facebook                                                                                                                                                                                                                                              0
Irrelevant                                                                                                                                                                         

In [10]:
# Rename columns for consistency
train_df.columns = ['id', 'topic', 'sentiment', 'text']
val_df.columns = ['id', 'topic', 'sentiment', 'text']

print("Training set after renaming:")
print(train_df[['sentiment', 'text']].head())

print("\nValidation set after renaming:")
print(val_df[['sentiment', 'text']].head())

# Check unique sentiment values
print("\nUnique sentiments in training:")
print(train_df['sentiment'].value_counts())

print("\nUnique sentiments in validation:")
print(val_df['sentiment'].value_counts())

Training set after renaming:
  sentiment                                               text
0  Positive  I am coming to the borders and I will kill you...
1  Positive  im getting on borderlands and i will kill you ...
2  Positive  im coming on borderlands and i will murder you...
3  Positive  im getting on borderlands 2 and i will murder ...
4  Positive  im getting into borderlands and i can murder y...

Validation set after renaming:
  sentiment                                               text
0   Neutral  BBC News - Amazon boss Jeff Bezos rejects clai...
1  Negative  @Microsoft Why do I pay for WORD when it funct...
2  Negative  CSGO matchmaking is so full of closet hacking,...
3   Neutral  Now the President is slapping Americans in the...
4  Negative  Hi @EAHelp I’ve had Madeleine McCann in my cel...

Unique sentiments in training:
sentiment
Negative      22542
Positive      20831
Neutral       18318
Irrelevant    12990
Name: count, dtype: int64

Unique sentiments in validation:
s

In [11]:
# Remove rows with missing text
print(f"Training set before removing missing: {len(train_df)}")
train_df = train_df.dropna(subset=['text'])
print(f"Training set after removing missing: {len(train_df)}")

print(f"Validation set before removing missing: {len(val_df)}")
val_df = val_df.dropna(subset=['text'])
print(f"Validation set after removing missing: {len(val_df)}")

Training set before removing missing: 74681
Training set after removing missing: 73995
Validation set before removing missing: 999
Validation set after removing missing: 999


In [17]:
# Twitter text cleaning function
def clean_twitter_text(text):
    if pd.isna(text):
        return ""
    
    text = str(text)
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'www\S+', '', text)
    
    # Remove user mentions but keep text
    text = re.sub(r'@\w+', '', text)
    
    # Handle emojis - convert to text description
    text = emoji.demojize(text)
    
    # Remove special characters but keep basic punctuation
    text = re.sub(r'[^\w\s!?,.]', '', text)
    
    # Handle repeated characters (e.g., "loooove" -> "love")
    text = re.sub(r'(.)\1+', r'\1\1', text)
    
    return text.strip().lower()

# Apply cleaning
print("Cleaning texts...")
train_texts = train_df['text'].apply(clean_twitter_text)
val_texts = val_df['text'].apply(clean_twitter_text)

# Prepare labels for 4-class classification
le = LabelEncoder()
le.fit(train_df['sentiment'])
y_train = le.transform(train_df['sentiment'])
y_val = le.transform(val_df['sentiment'])

print(f"Label mapping: {dict(zip(le.classes_, range(len(le.classes_))))}")
print(f"Training labels distribution: {np.bincount(y_train)}")
print(f"Validation labels distribution: {np.bincount(y_val)}")

Cleaning texts...
Label mapping: {'Irrelevant': 0, 'Negative': 1, 'Neutral': 2, 'Positive': 3}
Training labels distribution: [12875 22358 18108 20654]
Validation labels distribution: [171 266 285 277]


In [18]:
# Tokenization with larger vocabulary for Twitter data
tokenizer = Tokenizer(num_words=15000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_texts)

# Convert to sequences
train_sequences = tokenizer.texts_to_sequences(train_texts)
val_sequences = tokenizer.texts_to_sequences(val_texts)

# Analyze sequence lengths
train_lens = [len(seq) for seq in train_sequences]
print(f"Sequence length analysis:")
print(f"Max length: {max(train_lens)}")
print(f"Average length: {np.mean(train_lens):.2f}")
print(f"95th percentile: {np.percentile(train_lens, 95)}")

# Set max length based on analysis
max_length = 60  # Covers most tweets
X_train = pad_sequences(train_sequences, maxlen=max_length, padding='post')
X_val = pad_sequences(val_sequences, maxlen=max_length, padding='post')

print(f"\nFinal data shapes:")
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_val: {X_val.shape}, y_val: {y_val.shape}")
print(f"Vocabulary size: {len(tokenizer.word_index)}")

Sequence length analysis:
Max length: 166
Average length: 18.48
95th percentile: 47.0

Final data shapes:
X_train: (73995, 60), y_train: (73995,)
X_val: (999, 60), y_val: (999,)
Vocabulary size: 37107


In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

def create_lstm_model(vocab_size, num_classes=4, embedding_dim=100, lstm_units=64):
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=max_length),
        Bidirectional(LSTM(lstm_units, return_sequences=True)),
        Dropout(0.4),
        Bidirectional(LSTM(lstm_units//2)),
        Dropout(0.4),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Create model
vocab_size = min(len(tokenizer.word_index) + 1, 15000)
num_classes = len(le.classes_)

print(f"Creating model for {num_classes} classes...")
model = create_lstm_model(vocab_size, num_classes)
model.summary()

Creating model for 4 classes...


/home/chris/miniconda3/envs/deepclass/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025-10-02 20:18:53.514147: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [22]:
# Calculate class weights for imbalanced data
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weight_dict = dict(enumerate(class_weights))

print("Class weights for handling imbalance:")
for i, class_name in enumerate(le.classes_):
    print(f"  {class_name}: {class_weights[i]:.2f}")

# Callbacks
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=0.00001,
    verbose=1
)

# Train the model
print("Starting training...")
history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=64,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, reduce_lr],
    class_weight=class_weight_dict,
    verbose=1
)

Class weights for handling imbalance:
  Irrelevant: 1.44
  Negative: 0.83
  Neutral: 1.02
  Positive: 0.90
Starting training...
Epoch 1/30
1157/1157 ━━━━━━━━━━━━━━━━━━━━ 287s 232ms/step - accuracy: 0.4939 - loss: 1.1502 - val_accuracy: 0.8699 - val_loss: 0.4029 - learning_rate: 0.0010
Epoch 2/30
1157/1157 ━━━━━━━━━━━━━━━━━━━━ 292s 252ms/step - accuracy: 0.8201 - loss: 0.5135 - val_accuracy: 0.9109 - val_loss: 0.2534 - learning_rate: 0.0010
Epoch 3/30
1157/1157 ━━━━━━━━━━━━━━━━━━━━ 309s 267ms/step - accuracy: 0.8761 - loss: 0.3516 - val_accuracy: 0.9389 - val_loss: 0.2023 - learning_rate: 0.0010
Epoch 4/30
1157/1157 ━━━━━━━━━━━━━━━━━━━━ 261s 226ms/step - accuracy: 0.9008 - loss: 0.2729 - val_accuracy: 0.9459 - val_loss: 0.2089 - learning_rate: 0.0010
Epoch 5/30
1157/1157 ━━━━━━━━━━━━━━━━━━━━ 349s 302ms/step - accuracy: 0.9165 - loss: 0.2352 - val_accuracy: 0.9499 - val_loss: 0.1927 - learning_rate: 0.0010
Epoch 6/30
1157/1157 ━━━━━━━━━━━━━━━━━━━━ 301s 260ms/step - accuracy: 0.9253 - los